# 画像分類モデルを作ってみよう！

## Kerasとは

> Kerasは，Pythonで書かれた，TensorFlowまたはCNTK，Theano上で実行可能な高水準のニューラルネットワークライブラリです

公式ページより(https://keras.io/ja/#keras)

Kearsはいくつかのデータセットを用意してくれています。

- 画像分類
    - MNIST・・・手書き数字(0~9)が70000枚。
    - CIFAR10・・・10種類(airplane, automobile, bird, cat, deer, dog, frog, horse, ship, truck)の画像が60000枚。
    - CIFAR100・・・100種類の画像が60000枚。
    - Fashion-MNIST・・・服の白黒画像70000枚が10種類の服の種類にラベル付けされている。
- 自然言語処理
    - IMDB・・・映画レビュー感情分類。
    - ロイターニュース・・・ロイターニュースの記事にトピックがラベル付けされている。
- 回帰
    - StatLib・・・ボストン近郊の異なる住宅に関する10の属性値。その属性値から住宅価格を予測する。

本日はこの中から、CIFAR10を使用して画像の分類モデルを作成します。

(イメージ)
![イメージ](images/001.png)

## 流れ

1. データを用意する。
2. モデルを構築する。
3. モデルにデータを学習させる。
4. モデルを評価する。
5. 画像分類をやってみる。

### 1. データを用意する

CIFAR10のデータセットをダウンロードし、学習用データを表示してみましょう。

下のセルを1つ実行すると、この様なラベルと画像のセットが25個表示されます。

![イメージ](images/002.png)

ラベルの数字はそれぞれ以下の様な意味を持っており、この画像はラベルが "2" のため "鳥" の画像であることが確認できます。

|No|ラベル|
|:--|:--|
|0|飛行機|
|1|自動車|
|2|鳥|
|3|猫|
|4|鹿|
|5|犬|
|6|カエル|
|7|馬|
|8|船|
|9|トラック|

In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPool2D
from keras.layers.core import Dense,Activation,Dropout,Flatten
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np

#CIFAR10をダウンロード
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

#5x5枚の画像を表示する
plt.figure(figsize=(10,10))
for i in range(25):
    rand_num=np.random.randint(0,50000)
    cifar_img=plt.subplot(5,5,i+1)
    plt.imshow(x_train[rand_num])
    #目盛りを消す
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False, bottom=False, left=False, right=False, top=False)
    #正解ラベルを表示
    plt.title(y_train[rand_num])

plt.show()

#画像を0-1の範囲で正規化
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0

#正解ラベルをOne-Hot表現に変換
# 4 → [0,0,0,0,1,0,0,0,0,0]
# 0 → [1,0,0,0,0,0,0,0,0,0]
# 8 → [0,0,0,0,0,0,0,0,1,0]
y_train=np_utils.to_categorical(y_train,10)
y_test=np_utils.to_categorical(y_test,10)

### 2. モデルを構築する

次にモデルを構築していきましょう。

Kerasにはニューラルネットワークモデルを定義する方法が2種類あります。[(参考リンク)](https://keras.io/ja/models/about-keras-models/)

|種類|説明|レベル感|
|:--|:--|:--|
|Sequential モデル|層を積み重ねる最も単純で使いやすいモデル構築方法|初学者向け|
|Functional API モデル|関数を積み上げるように構築するAPIで、最も一般的に使われている|中級〜上級者向け|

本資料では**Sequential モデル**を利用します。

In [ ]:
#モデルを構築
model=Sequential()

#層の定義
model.add(Conv2D(32,(3,3),padding='same',input_shape=(32,32,3)))
model.add(Activation('relu'))
model.add(Conv2D(32,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64,(3,3),padding='same'))
model.add(Activation('relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(10,activation='softmax'))

#モデルをコンパイル
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

### 3. モデルにデータを学習させる

作成したモデルに対して学習用データを入力し、学習をさせます。

1つ下のセルを実行すると、学習の進捗を以下の様なログで確認することができます。

> ```log
> Epoch 1/5
> 352/352 [==============================] - 88s 250ms/step - loss: 1.0494 - accuracy: 0.6250 - val_loss: 0.9374 - val_accuracy: 0.6688
> Epoch 2/5
> 352/352 [==============================] - 87s 247ms/step - loss: 0.9396 - accuracy: 0.6673 - val_loss: 0.8097 - val_accuracy: 0.7184
> ・
> ・
> ・
> ```

果たして満足のいく精度が出せるでしょうか…

In [ ]:
#学習
history=model.fit(x_train,y_train,batch_size=128,epochs=5,verbose=1,validation_split=0.1)

#モデルの表示
model.summary()

### 4. モデルを評価する

学習が完了したら、テスト用データを入力してモデルを評価しましょう。

"Test accuracy" に注目です！

1に近いほど精度の高いモデルができたことになります。

In [ ]:
#評価
score=model.evaluate(x_test,y_test,verbose=0)
print('Test loss:',score[0])
print('Test accuracy:',score[1])

### 5. 画像分類をやってみる

精度を確認したら、実際に画像を渡して正解できるか確認をしてみます

In [ ]:
#model.predict()で画像のクラスを予想する,　最も予測結果のスコアの高いラベルと正解のラベルを取得
img_pred=np.argmax(model.predict(x_test), axis=-1)
img_test=np.argmax(y_test, axis=-1)

#2x2枚の画像を表示する
plt.figure(figsize=(10,10))
for i in range(4):
    rand_num=np.random.randint(0,10000)
    cifar_img=plt.subplot(2,2,i+1)
    plt.imshow(x_test[rand_num])
    #目盛りを消す
    plt.tick_params(labelbottom=False, labelleft=False, labelright=False, labeltop=False, bottom=False, left=False, right=False, top=False)
    #画像の予想
    plt.title('pred:{0}, ans:{1}'.format(img_pred[rand_num],img_test[rand_num]))

plt.show()

任意の画像をUPLOADして分類してみます

In [ ]:
#画像読み込み
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
from keras.preprocessing.image import img_to_array, load_img
from io import BytesIO
from PIL import Image

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='xxxxxx',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.ap.cloud-object-storage.appdomain.cloud')

bucket = 'xxxxxx'
object_key = 'xxxxxx.jpeg'

#ここを追記objectを画像に変換
temp_img = cos_client.get_object(Bucket=bucket, Key=object_key)['Body'].read()
image = Image.open(BytesIO(temp_img))
img_resize = image.resize((32, 32),Image.LANCZOS)

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/

#画像を配列に変換し0-1で正規化
img_array=img_to_array(img_resize)
img_array=img_array.astype('float32')/255.0
img_array=img_array.reshape((1,32,32,3))

#画像を予想
img_pred=np.argmax(model.predict(img_array), axis=-1)

plt.imshow(image)
plt.title('pred:{}'.format(img_pred))
plt.show()